<a href="https://colab.research.google.com/github/ahmoura/icsme2020-replication-package/blob/master/Finding_bugs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm

In [6]:
import pandas as pd
import numpy as np

import scipy.stats as stats
# Graduandos:
# No modelo:
#   Remover sem nome (index);
#   Aplicar Random Search (descobrir features importantes);
#   Se der, aplicar uma regressão logística;
#   Estudar AUC e ROC,se possível aplicar ao modelo;
#   Precision e Recall;
#   Aplicar correlação pela direita e esquerda:
#   Correlação; 
#   Média;
#   Quantidade de cada arquivo, commit, etc.
#   Detecção de Anomalias


# elemento no X pode estar vazando atributo alvo.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
df_simple = pd.read_csv("/content/drive/My Drive/dataset_icsme/simple_data.csv")
df_replication = pd.read_csv("/content/drive/My Drive/dataset_icsme/replication_data.csv")

In [11]:
df_simple

,Unnamed: 0,files,authors,commits,days,conflicts,complex,bic
0,37,161,10,69,24,1,True,False
1,106,17,3,4,3,1,False,False
2,138,307,29,57,4,1,True,True
3,141,207,20,50,12,1,True,False
4,154,881,25,77,5,9,True,True
...,...,...,...,...,...,...,...,...
1585,21148,55,2,6,5,9,True,False
1586,21162,31,5,30,6,1,False,False
1587,21166,10,2,10,2,1,False,False
1588,21178,19,2,6,3,1,False,False


In [12]:
df_replication

,Unnamed: 0,Name,Merge,Base,right,left,a_mergeDate,c_mergeDate,a_baseDate,c_baseDate,a_rightDate,c_rightDate,a_leftDate,c_leftDate,rightFiles,rightAdditions,rightRemovals,leftFiles,leftAdditions,leftRemovals,nCommitsRight,authorsRight,nCommitsLeft,authorsLeft,conflict
0,20,activemq,3870aa94473b2253e7a38e5897187d1049d60059,af999fe2b2f5140fd7fcae187b491dddbcde1fe9,aeb93e4a2455dff169da302fb19f7fa55d21a0c0,002ade79b01db228377c24438bae246690f57b7d,2015-06-29 08:34:50 +0200,2015-06-29 08:34:50 +0200,2015-06-05 18:05:03 -0400,2015-06-05 18:05:03 -0400,2015-06-06 17:02:31 +0200,2015-06-06 17:02:31 +0200,2015-06-06 17:02:31 +0200,2015-06-06 17:02:31 +0200,3,346,75,158,12992,628,10,1,59,9,1
1,41,activemq,f74a65f4f5e712051c33ed7b44605dc3cbb93f7f,da3775302c68c628138a221409910ff9c4ca48a6,f222741fdc5e5b8b5dc401e054abbf5a77ff63dc,127f82a4392caeec2913144a72a61f53c79d5625,2014-02-03 15:23:54 -0500,2014-02-03 15:23:54 -0500,2014-01-31 17:23:21 -0500,2014-01-31 17:25:26 -0500,2014-02-03 15:13:44 -0500,2014-02-03 15:13:44 -0500,2014-02-03 15:13:44 -0500,2014-02-03 15:13:44 -0500,7,53,28,10,301,0,2,2,2,1,1
2,113,ambari,d222f572ee2952add29e60afeb47a4da7ac98685,d6d247983c5c66c11260a6d10e7b0e2746fa8146,393ab3ee28a27c63084ee4700bf0386b6ceb4a9b,132e266521642d90908d3dd69446a701ca94c3db,2017-02-03 09:52:46 -0500,2017-02-03 09:52:46 -0500,2017-01-30 15:54:20 +0100,2017-01-30 15:54:20 +0100,2017-01-30 10:40:13 -0500,2017-01-30 10:40:13 -0500,2017-01-30 10:40:13 -0500,2017-01-30 10:40:13 -0500,39,818,137,377,6192,2255,36,1,73,33,2
3,114,ambari,3bd91a2f7968cb182d005a3b45879261297b4495,358a58871745fb65e93f1e968c03141ee9c6368a,56f838b5b0a5681dffb703f503fcf443cb3b6dbd,f516b6c5ad384ceb52181bee6b1553089563182e,2017-04-19 11:04:49 -0400,2017-04-19 11:04:49 -0400,2017-04-12 17:35:29 -0700,2017-04-17 11:47:26 -0700,2017-04-17 17:19:45 -0400,2017-04-18 14:13:21 -0400,2017-04-17 17:19:45 -0400,2017-04-18 14:13:21 -0400,43,1218,457,63,372,255,53,2,5,4,1
4,115,ambari,e3d5344a6cf0e22455580cd9a63f8e30256c7097,e5fff5825563bee0b0af7a0fd790731989b9ce0b,c8858af27daa31bcfc1bef92f6630483c6a1dc41,6a0b2a0e2be6901b13db0787fa70ff98905b19da,2017-04-26 13:59:54 -0400,2017-04-26 13:59:54 -0400,2017-04-20 15:29:46 -0700,2017-04-23 15:21:53 -0700,2017-04-23 22:19:59 -0400,2017-04-23 22:19:59 -0400,2017-04-23 22:19:59 -0400,2017-04-23 22:19:59 -0400,43,1218,457,137,3682,590,56,2,33,21,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,21156,zeppelin,8763fa8a74d122a3e51e3208f818450c48810464,9461cf20c0fb93b93e0a264f245913f9d62d01c2,8d84c513dfc7e36b9bd7a4de10dc342688cda660,52c416623d15f9275c3647186687f427b74e609b,2013-12-20 03:03:34 +0900,2013-12-20 03:03:34 +0900,2013-12-13 23:23:34 -0800,2013-12-13 23:23:34 -0800,2013-12-17 01:23:32 +0900,2013-12-17 01:23:32 +0900,2013-12-17 01:23:32 +0900,2013-12-17 01:23:32 +0900,4,258,1,18,1587,1254,2,1,12,2,2
1586,21165,zeppelin,74850964a93ae07e0cbda90a35dd1303bb98a50a,00775ed4a10eb34659dd02c1372655ea9b482592,27748281918d5d708fb823a5a2991eae08dfd196,f5b4175f2e090cff520c5b7f69679c968a718133,2013-10-29 08:47:47 +0900,2013-10-29 08:47:47 +0900,2013-10-28 20:55:02 +0900,2013-10-28 20:55:02 +0900,2013-10-29 08:04:42 +0900,2013-10-29 08:04:42 +0900,2013-10-29 08:04:42 +0900,2013-10-29 08:04:42 +0900,5,6,6,5,6,6,1,1,1,1,5
1587,21170,zeppelin,d5c31bb03d941f15ea5ddef52f9979d56139d366,80b4f73fa4f5a7adb53eaf5abc41e4586437be8d,0f48808a4a501d5b86bf32708768bd629edc7fba,555cec4fc75c6d2d62450afbe7e3d45ebe2e07b3,2013-10-16 23:00:55 +0900,2013-10-16 23:00:55 +0900,2013-10-11 18:18:49 +0900,2013-10-11 18:18:49 +0900,2013-10-15 23:44:51 +0900,2013-10-15 23:44:51 +0900,2013-10-15 23:44:51 +0900,2013-10-15 23:44:51 +0900,14,251,121,41,326,2363,1,1,5,1,9
1588,21172,zeppelin,a4153c9f0b788c4c83be28a46e81e5170a30cba9,628c7fb1f0d4d4946442e56333e90ede02ca6edd,23d2577130311e08c7f12388fea90325c7c1e91d,7cd3d5725774c4fc7b3b48f6f08b57b9fc8534cc,2014-01-02 22:53:48 +0900,2014-01-02 22:53:48 +0900,2013-12-21 00:02:05 -0800,201

In [13]:
df = pd.concat([df_simple,df_replication], axis=1)
df

,Unnamed: 0,files,authors,commits,days,conflicts,complex,bic,Unnamed: 0,Name,Merge,Base,right,left,a_mergeDate,c_mergeDate,a_baseDate,c_baseDate,a_rightDate,c_rightDate,a_leftDate,c_leftDate,rightFiles,rightAdditions,rightRemovals,leftFiles,leftAdditions,leftRemovals,nCommitsRight,authorsRight,nCommitsLeft,authorsLeft,conflict
0,37,161,10,69,24,1,True,False,20,activemq,3870aa94473b2253e7a38e5897187d1049d60059,af999fe2b2f5140fd7fcae187b491dddbcde1fe9,aeb93e4a2455dff169da302fb19f7fa55d21a0c0,002ade79b01db228377c24438bae246690f57b7d,2015-06-29 08:34:50 +0200,2015-06-29 08:34:50 +0200,2015-06-05 18:05:03 -0400,2015-06-05 18:05:03 -0400,2015-06-06 17:02:31 +0200,2015-06-06 17:02:31 +0200,2015-06-06 17:02:31 +0200,2015-06-06 17:02:31 +0200,3,346,75,158,12992,628,10,1,59,9,1
1,106,17,3,4,3,1,False,False,41,activemq,f74a65f4f5e712051c33ed7b44605dc3cbb93f7f,da3775302c68c628138a221409910ff9c4ca48a6,f222741fdc5e5b8b5dc401e054abbf5a77ff63dc,127f82a4392caeec2913144a72a61f53c79d5625,2014-02-03 15:23:54 -0500,2014-02-03 15:23:54 -0500,2014-01-31 17:23:21 -0500,2014-01-31 17:25:26 -0500,2014-02-03 15:13:44 -0500,2014-02-03 15:13:44 -0500,2014-02-03 15:13:44 -0500,2014-02-03 15:13:44 -0500,7,53,28,10,301,0,2,2,2,1,1
2,138,307,29,57,4,1,True,True,113,ambari,d222f572ee2952add29e60afeb47a4da7ac98685,d6d247983c5c66c11260a6d10e7b0e2746fa8146,393ab3ee28a27c63084ee4700bf0386b6ceb4a9b,132e266521642d90908d3dd69446a701ca94c3db,2017-02-03 09:52:46 -0500,2017-02-03 09:52:46 -0500,2017-01-30 15:54:20 +0100,2017-01-30 15:54:20 +0100,2017-01-30 10:40:13 -0500,2017-01-30 10:40:13 -0500,2017-01-30 10:40:13 -0500,2017-01-30 10:40:13 -0500,39,818,137,377,6192,2255,36,1,73,33,2
3,141,207,20,50,12,1,True,False,114,ambari,3bd91a2f7968cb182d005a3b45879261297b4495,358a58871745fb65e93f1e968c03141ee9c6368a,56f838b5b0a5681dffb703f503fcf443cb3b6dbd,f516b6c5ad384ceb52181bee6b1553089563182e,2017-04-19 11:04:49 -0400,2017-04-19 11:04:49 -0400,2017-04-12 17:35:29 -0700,2017-04-17 11:47:26 -0700,2017-04-17 17:19:45 -0400,2017-04-18 14:13:21 -0400,2017-04-17 17:19:45 -0400,2017-04-18 14:13:21 -0400,43,1218,457,63,372,255,53,2,5,4,1
4,154,881,25,77,5,9,True,True,115,ambari,e3d5344a6cf0e22455580cd9a63f8e30256c7097,e5fff5825563bee0b0af7a0fd790731989b9ce0b,c8858af27daa31bcfc1bef92f6630483c6a1dc41,6a0b2a0e2be6901b13db0787fa70ff98905b19da,2017-04-26 13:59:54 -0400,2017-04-26 13:59:54 -0400,2017-04-20 15:29:46 -0700,2017-04-23 15:21:53 -0700,2017-04-23 22:19:59 -0400,2017-04-23 22:19:59 -0400,2017-04-23 22:19:59 -0400,2017-04-23 22:19:59 -0400,43,1218,457,137,3682,590,56,2,33,21,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1585,21148,55,2,6,5,9,True,False,21156,zeppelin,8763fa8a74d122a3e51e3208f818450c48810464,9461cf20c0fb93b93e0a264f245913f9d62d01c2,8d84c513dfc7e36b9bd7a4de10dc342688cda660,52c416623d15f9275c3647186687f427b74e609b,2013-12-20 03:03:34 +0900,2013-12-20 03:03:34 +0900,2013-12-13 23:23:34 -0800,2013-12-13 23:23:34 -0800,2013-12-17 01:23:32 +0900,2013-12-17 01:23:32 +0900,2013-12-17 01:23:32 +0900,2013-12-17 01:23:32 +0900,4,258,1,18,1587,1254,2,1,12,2,2
1586,21162,31,5,30,6,1,False,False,21165,zeppelin,74850964a93ae07e0cbda90a35dd1303bb98a50a,00775ed4a10eb34659dd02c1372655ea9b482592,27748281918d5d708fb823a5a2991eae08dfd196,f5b4175f2e090cff520c5b7f69679c968a718133,2013-10-29 08:47:47 +0900,2013-10-29 08:47:47 +0900,2013-10-28 20:55:02 +0900,2013-10-28 20:55:02 +0900,2013-10-29 08:04:42 +0900,2013-10-29 08:04:42 +0900,2013-10-29 08:04:42 +0900,2013-10-29 08:04:42 +0900,5,6,6,5,6,6,1,1,1,1,5
1587,21166,10,2,10,2,1,False,False,21170,zeppelin,d5c31bb03d941f15ea5ddef52f9979d56139d366,80b4f73fa4f5a7adb53eaf5abc41e4586437be8d,0f48808a4a501d5b86bf32708768bd629edc7fba,555cec4fc75c6d2d62450afbe7e3d45ebe2e07b3,2013-10-16 23:00:55 +0900,2013-10-16 23:00:55 +0900,2013-10-11 18:18:49 +0900,2013-10-11 18:18:49 +0900,2013-10-15 23:44:51 +0900,2013-10-15 23:44:51 +0900,2013-10-15 23:44:51 +0900,20

In [14]:
df = df.drop(["a_mergeDate",	"c_mergeDate",	"Unnamed: 0", "a_baseDate",	"c_baseDate",	"a_rightDate", "c_rightDate",	"a_leftDate",	"c_leftDate", "Merge", "Base", "right", "left", "Name","Unnamed: 0"], axis = 1)

In [15]:
df.dtypes

files             int64
authors           int64
commits           int64
days              int64
conflicts         int64
complex            bool
bic                bool
rightFiles        int64
rightAdditions    int64
rightRemovals     int64
leftFiles         int64
leftAdditions     int64
leftRemovals      int64
nCommitsRight     int64
authorsRight      int64
nCommitsLeft      int64
authorsLeft       int64
conflict          int64
dtype: object

In [16]:
df_simple[df_simple["bic"] == True]

,Unnamed: 0,files,authors,commits,days,conflicts,complex,bic
2,138,307,29,57,4,1,True,True
4,154,881,25,77,5,9,True,True
8,164,306,25,107,5,7,True,True
12,194,984,55,371,3,6,True,True
14,200,2243,13,40,1,12,True,True
...,...,...,...,...,...,...,...,...
1512,20441,177,17,189,53,6,True,True
1522,20545,99,5,39,40,3,True,True
1544,20769,1072,12,729,260,2,True,True
1552,20829,26,3,13,3,2,False,True


In [17]:
df.bic.value_counts() / df.shape[0]

False    0.90566
True     0.09434
Name: bic, dtype: float64

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import cohen_kappa_score, make_scorer

In [19]:
X = df.drop(columns=["bic"])
y = df["bic"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.3, stratify=y)

In [20]:
clf = LGBMClassifier(random_state=0, class_weight='balanced')
clf.fit(X_train, y_train)
predict = clf.predict(X_test)

In [21]:
score_rf = cohen_kappa_score(clf.predict(X_test),y_test)

score_rf

-0.014521452145214608

In [23]:
X = df.drop("bic", axis = 1)
y = df["bic"]

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0, test_size = 0.3)

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import cohen_kappa_score, make_scorer


cohen_score =  make_scorer(cohen_kappa_score)
distributions = {'criterion':['gini','entropy'],
                'max_depth': stats.randint(2, 8),
                'max_features': stats.randint(5,20),
                'min_samples_leaf': stats.randint(3, 7),
                'min_samples_split': stats.randint(8, 10),
                'n_estimators': stats.randint(100,1000)}
logistic = RandomForestClassifier()
# cv = n-cross-validation
random_search = RandomizedSearchCV(logistic, distributions, random_state=0, verbose=1, n_jobs=-1,n_iter=50, scoring=cohen_score)


random_search.fit(X_train, y_train)

print('Melhores parametros = \n',random_search.best_params_)
print('\nMelhor pontuação: \n',random_search.best_score_)
#{'criterion': 'entropy', 'max_depth': 5, 'max_features': 17, 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 815}
#{'criterion': 'entropy', 'max_depth': 7, 'max_features': 11, 'min_samples_leaf': 5, 'min_samples_split': 8, 'n_estimators': 173}

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  6.1min finished


Melhores parametros = 
 {'criterion': 'gini', 'max_depth': 7, 'max_features': 5, 'min_samples_leaf': 6, 'min_samples_split': 9, 'n_estimators': 807}

Melhor pontuação: 
 0.0


In [25]:

feature_importantes = pd.DataFrame(random_search.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importantes)
y_score = random_search.predict_proba(X_test)

                importance
files             0.134321
leftAdditions     0.095649
rightRemovals     0.074927
rightFiles        0.071934
rightAdditions    0.068982
commits           0.065362
leftRemovals      0.065324
leftFiles         0.061480
conflicts         0.055471
nCommitsLeft      0.050657
authors           0.050139
nCommitsRight     0.049116
days              0.048815
conflict          0.036683
authorsLeft       0.036313
authorsRight      0.030824
complex           0.004004


In [29]:
from sklearn.metrics import roc_auc_score, roc_curve
a = [1 if x[1]>=.5 else 0 for x in y_score]

In [28]:
print(roc_auc_score(y_test, a))

0.5


In [31]:
roc_curve(y_test, a)

(array([0., 1.]), array([0., 1.]), array([1, 0]))